In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from multiprocessing.pool import ThreadPool
from multiprocessing import cpu_count
max_cpu = cpu_count()

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#### Part (a)



#### Part (b)



#### Part (c)



#### Part (d)



In [3]:
def DGP():
    Z = np.random.uniform(0,1,size=(500,10))
    epsilon, nu = np.random.normal(0,0.25,size=(500,1)), np.random.normal(0,0.25,size=(500,1))
    g_z = np.multiply(Z[:,1],np.add(Z[:,2],Z[:,3]))
    X = g_z.reshape(500,1) + nu
    m_xz = np.multiply(X,(np.add(Z[:,1],Z[:,2]) ** 2).reshape(500,1))
    y = m_xz.reshape(500,1) + epsilon

    gamma = np.random.normal(size=(10,1))
    p_z = np.exp(-Z @ gamma) / (1 + np.exp(-Z @ gamma))
    zeta = np.random.uniform(0,1)
    D = np.where((p_z - zeta) > 0,1,0).reshape(500,1)
    
    return np.concatenate((y,X,Z,D),axis=1)

def calc_theta(Y,X,D,phi,pi,prop):
    num = ((X * Y * D) / prop) - ((D - prop) * (phi / prop))
    e_num = np.sum(num) / num.shape[0]
    denom = (((X ** 2) * D) / prop)  - ((D - prop) * (pi / prop))
    e_denom = np.sum(denom) / denom.shape[0]
    theta_hat = (e_num / e_denom)
    
    return theta_hat

def calc_J(X,D,pi,prop):
    J = (1 / prop) * ((X ** 2) * D - (D - prop) * pi)
    e_J = np.sum(J) / J.shape[0]
    
    return e_J

def calc_g(Y,X,D,phi,pi,prop,theta):
    left = ((((X ** 2) * D) / prop) - ((D - prop) * (pi / prop))) * theta
    right = ((X * Y * D) / prop) + ((D - prop) * (phi / prop))
    g = left - right
    e_g = np.sum(g) / g.shape[0]
    
    return e_g

def cv(data,k=5):
    I_sections = np.array_split(data,5)

    thetas = []
    gs = []
    for i,_ in enumerate(I_sections):
        I_k = I_sections[i]
        I_minusk_sections = I_sections[:i] + I_sections[i+1:]
        I_minusk = np.concatenate(I_minusk_sections,axis=0)
        
        # estimating theta_k
        ## estimating nuisance parameters
        Z_minusk = I_minusk[:,3:13]
        data_D1_minusk = I_minusk[I_minusk[:,-1] == 1]
        Z_D1_minusk = I_minusk[:,3:13]

        phi_rf = RandomForestRegressor(random_state=14)
        phi_cv = GridSearchCV(phi_rf, n_jobs = -1,
                              param_grid = {'max_features':np.arange(1,11)})
        phi_cv.fit(Z_D1_minusk,I_minusk[:,0] * I_minusk[:,1])

        pi_rf = RandomForestRegressor(random_state=14)
        pi_cv = GridSearchCV(pi_rf, n_jobs = -1,
                             param_grid = {'max_features':np.arange(1,11)})
        pi_cv.fit(Z_D1_minusk,I_minusk[:,1] ** 2)

        prop_rf = RandomForestClassifier(random_state=14)
        prop_cv = GridSearchCV(prop_rf, n_jobs = -1,
                               param_grid = {'max_features':np.arange(1,11)})
        prop_cv.fit(Z_minusk,I_minusk[:,-1])


        ## calculate theta_k
        Z_k = I_k[:,3:13].reshape(I_k.shape[0],10)

        Y_k = I_k[:,0].reshape(I_k.shape[0],1)
        X_k = I_k[:,1].reshape(I_k.shape[0],1)
        D_k = I_k[:,-1].reshape(I_k.shape[0],1)

        phi_hat = phi_cv.predict(Z_k).reshape(Z_k.shape[0],1)
        pi_hat = pi_cv.predict(Z_k).reshape(Z_k.shape[0],1)
        temp = np.where(prop_cv.predict_proba(Z_k)[:,1] == 0, 0.001, prop_cv.predict_proba(Z_k)[:,1])
        prop_hat = np.where(temp == 1, 0.999, temp).reshape(Z_k.shape[0],1)

        theta_k = calc_theta(Y_k,X_k,D_k,phi_hat,pi_hat,prop_hat)
        thetas.append(theta_k)
        
        # variance estimator
        g = calc_g(Y_k,X_k,D_k,phi_hat,pi_hat,prop_hat,theta_k)
        gs.append(g)        
        
    theta_hat = sum(thetas) / k
    g = sum(gs) / k
    
    ## calculating J
    phi_rf = RandomForestRegressor(random_state=14)
    phi_cv = GridSearchCV(phi_rf, n_jobs = -1,
                          param_grid = {'max_features':np.arange(1,11)})
    phi_cv.fit(data[data[:,-1] == 1][:,3:13],
               data[data[:,-1] == 1][:,0] * data[data[:,-1] == 1][:,1])

    pi_rf = RandomForestRegressor(random_state=14)
    pi_cv = GridSearchCV(pi_rf, n_jobs = -1,
                         param_grid = {'max_features':np.arange(1,11)})
    pi_cv.fit(data[data[:,-1] == 1][:,3:13],data[data[:,-1] == 1][:,1] ** 2)

    prop_rf = RandomForestClassifier(random_state=14)
    prop_cv = GridSearchCV(prop_rf, n_jobs = -1,
                           param_grid = {'max_features':np.arange(1,11)})
    prop_cv.fit(data[:,3:13],data[:,-1])
    
    phi_hat = phi_cv.predict(data[:,3:13])
    pi_hat = pi_cv.predict(data[:,3:13])
    temp = np.where(prop_cv.predict_proba(data[:,3:13])[:,1] == 0, 
                    0.001, 
                    prop_cv.predict_proba(data[:,3:13])[:,1])
    prop_hat = np.where(temp == 1, 0.999, temp).reshape(data[:,3:13].shape[0],1)
    
    X = data[:,1].reshape(data.shape[0],1)
    D = data[:,-1].reshape(data.shape[0],1)
    J = calc_J(X,D,pi_hat,prop_hat)
    
    ## calculating variance
    var_hat = (1 / J) * (g ** 2) * (1 / J)
    
    return theta_hat, var_hat

In [4]:
data = DGP()

In [5]:
theta_hat, var_hat = cv(data)

In [6]:
theta_hat

1.9009819308208555

In [8]:
var_hat

4.715738530596786e-05

In [10]:
se = np.sqrt(var_hat / data.shape[0])
se

0.0003071070995791789

In [11]:
# 95% CI

print(theta_hat - se, theta_hat + se)

1.9006748237212763 1.9012890379204348
